In [1]:
import cv2
import numpy as np
from arucoprocessor import ArucoProcessor
from tqdm import tqdm
import plotly.graph_objects as go

In [2]:
cap = cv2.VideoCapture('aruco_video.mp4')

frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

fc = 0
ret = True

while (fc < frameCount and ret):
    ret, buf[fc] = cap.read()
    fc += 1

cap.release()

In [3]:
positions = []
orientations = []
apc = ArucoProcessor()

for i in tqdm(range(150)):
    if apc.process_frame(buf[i]):
        pose = apc.get_aruco_pose()
        positions.append(pose[:3, 3])
        orientations.append(pose[:3, :3])

positions = np.array(positions)

  1%|▏         | 2/150 [00:00<00:10, 13.98it/s]

  3%|▎         | 4/150 [00:00<00:10, 14.39it/s]

Reliable


100%|██████████| 150/150 [00:08<00:00, 18.60it/s]


In [4]:
layout = go.Layout(
             scene=dict(
                 aspectmode='data'
         ))

fig = go.Figure(data=[go.Scatter3d(x=positions.T[0], y=positions.T[1], z=positions.T[2],
                                    marker_size=5, text=np.arange(0, len(positions)), line=dict(
        color='darkblue',
        width=2
    )
)], layout=layout)

fig.show()

In [9]:
# fig.write_html("x_axis_interp.html")

In [13]:
x_unit = np.array([[1],[0],[0]]) * 0.04
y_unit = np.array([[0],[1],[0]]) * 0.04
z_unit = np.array([[0],[0],[1]]) * 0.04

x_ends = []
y_ends = []
z_ends = []

for i in range(len(orientations)):
    x_ends.append(orientations[i] @ x_unit)
    y_ends.append(orientations[i] @ y_unit)
    z_ends.append(orientations[i] @ z_unit)

layout = go.Layout(scene=dict(aspectmode='data'))

arrows = []

for i in range(len(x_ends)):
    arrows.append(go.Scatter3d(x=[positions[i][0], positions[i][0] + x_ends[i][0][0]],
                               y=[positions[i][1], positions[i][1] + x_ends[i][1][0]],
                               z=[positions[i][2], positions[i][2] + x_ends[i][2][0]],
                               marker_size=1, line=dict(color='red',width=2)))

for i in range(len(y_ends)):
    arrows.append(go.Scatter3d(x=[positions[i][0], positions[i][0] + y_ends[i][0][0]],
                               y=[positions[i][1], positions[i][1] + y_ends[i][1][0]],
                               z=[positions[i][2], positions[i][2] + y_ends[i][2][0]],
                               marker_size=1, line=dict(color='green',width=2)))

for i in range(len(z_ends)):
    arrows.append(go.Scatter3d(x=[positions[i][0], positions[i][0] + z_ends[i][0][0]],
                               y=[positions[i][1], positions[i][1] + z_ends[i][1][0]],
                               z=[positions[i][2], positions[i][2] + z_ends[i][2][0]],
                               marker_size=1, line=dict(color='blue',width=2)))

fig = go.Figure(data=[go.Scatter3d(x=positions.T[0], y=positions.T[1], z=positions.T[2],
                                    marker_size=3, text=np.arange(0, len(positions)), line=dict(color='black',width=2))] + arrows,
                                    layout=layout)

fig.show()